# Risk Sensative Control

## Introdunction
Exisiting autonoumous driving controls assume that the users of the autonoumous driving system prefer the similar pattern, in practical scenarios, this is rarely the case. This paper introduces Risk Sensative Control (RSC), an inverse optimal control algorithm that is able to estimate the risk sensative driving features, and incorporate those features online into a receding-horizon cotroller. RSC solves the inverse optimal control problem by reducing subjective risk perseption from users feedback. RSC uses an Meta-learning algorithm to update the parameters of the cost function parameters, and  improve the controller online as more data is gathered from the users subjective risk feedback. The estimater take acounts of the uncertainty in the subjective risk analysis in driving features and surroudning vehicle locations, by uncertainty ellipse constraints. This approach is verified in realtime in five lane change both safe and risky scenarios in CARLA simulation environments.


## Related Work

### Subjective Risk Estimation

### Objective Function Estimation
Estimating objective functions from historical data is a well investigated problem known as Inverse Optimal Control
(IOC) or Inverse Reinforcement Learning (IRL). Typically, with the IRL approach, the objective function is linear in
terms of a given set of state features. The goal is to identify a parameter vector that weights these features so that
the behavior resulting from this estimated objective matches the observed behavior. 
Optimal control methods are optimizers, but in reality, we don’t really know what we should be optimizing. The question of how we can, instead, leverage our strong intuition of how we’d like the robot to behave to learn a cost function that embodies demonstrations of that behavior. So defines Inverse Optimal Control (IOC). Rather than computing an optimal policy given a cost function, IOC uncovers the cost function that best explains demonstrated optimal behavior. 

### Online Parameter Estimation
We review approaches proposed for online parameter estimation in this context. The Intelligent Driver Model (IDM), is a rule-based lane-following model balancing two objectives: reaching a desired speed and avoiding collision with
the preceding car. This model has few tunable parameters and returns the scalar acceleration of the vehicle along a predefined
path. Several works used online filtering techniques to estimate these IDM parameters.


### Data-Driven driving Control
There is a rich literature on applying data-driven approaches
to pedestrian or vehicle trajectory predictions. Such approaches usually require a large corpus of data and are trained
offline as a general model to suit multiple agents. On the other hand, we require a small amount of data and find parameters
online for a specific agent. Data-driven methods can predict a distribution over future trajectories e.g., offline inverse optimal
control with online goal inference; Conditional Variational Autoencoders (CVAE) or Generative Adversarial Networks (GAN). 

## Problem formulation




- Ego vehicle state: $$z=\begin{bmatrix}x\\ y\\ \phi\\ v\\ \omega\\  l \\ \dot{a} \\ \ddot{a}\end{bmatrix}$$

which represents $x, y$ position of trajectory, yaw angle, velocity, lateral acceleration, longitudinal acceleration, derivative of accel force, second order derivative of accel force, derivative of brake force, second order derivative of brake force.

- Ego vehicle control $$u = \begin{bmatrix}\delta\\a\end{bmatrix}$$

which represents steering angle, accel force, or brake force.

Our lane change problem can be defined as the following figure. Red vehicle is our ego vehicle we is supposed to lane change to the next lane to pass the preceding vehicle. We focus on the discretized dynamic state-space form setting within length $N$ time horizon. We denote $z_t \in R^n$ the state and $u_t \in R^m$ the control input of the ego vehicle in $t \in \{1,...,N\}$ time steps. Ego vehicle's strategy is a control input sequence $U = [u_{t+1}, u_{t+2}, ...,u_{t+N}]$ and $u_{t+1}$ will be applied in each loop. We donote $z_t^k, k \in {1,...,6}$ to represent the state of surrounding vehicle in $k$ direction relative to ego vehicle in $t$ time step, and $u_t^k$ to represent the control input of surrounding vehicles. We assume the surrounding vehicle to drive in the same manner of ego vehicle but in the default parameter setting without lane changing.


<img src=./fig/scenario_1.png width="500px" >


## Surrounding vehicle description


In a mylti-player driving situation, we assume the other agents are 'ideal' players. We further assume that ego vehicle and surrounding ego vehicles in a receding horizon loop.

<img src=./fig/problem_definition.png width="500px" >

## Features and Objectives

Since we use CARLA simulation environment to demonstrate our problem definition, we list up the features information to describe our system as following.

#### Ego vehicle state
| Features | Description | Carla.PythonAPI | Unit | Comment |
|----------|-------------|-----------------|------:|---------|
|      x   |x position of ego vehicle in the coordinate system   |  carla.Location.x |   m |         |
|      y   |y position of ego vehicl in the coordinate system   |  carla.Location.y |   m |         |
|      yaw   | yaw rotation Z-axis rotation angle.   |  carla.Transform.yaw |   degrees  |         |
|      v.x   | velocity vector    |  carla.get_velocity.x |   m/s  |         |
|      v.y   | velocity vector   |  carla.get_velocity.y |   m/s  |         |
|      v.z   | velocity vector    |  carla.get_velocity.z |   m/s  |         |
|      $\omega$.x   | angular velocity vector     |  carla.get_angular_velocity.x |   deg/s  |         |
|      $\omega$.y   | angular velocity vector     |  carla.get_angular_velocity.y |   deg/s  |         |
|      $\omega$.z   |angular velocity vector    |  carla.get_angular_velocity.z |   deg/s  |         |
|      $a$.x   |  longitudinal acceleration vector     |  carla.get_acceleration.x |   deg/s  |         |
|      $a$.y   | lateral acceleration vector     |  carla.get_acceleration.y |   deg/s  |         |
|      $a$.z   | acceleration vector   |  carla.get_acceleration.z |   deg/s  |         |


## Motion Model (New)

We focus on the discretized dynamic state-space form setting within length $N$ time horizon. We denote $z_t \in R^n$ the state and $u_t \in R^m$ the control input of the ego vehicle in $t \in \{1,...,N\}$ time steps. Ego vehicle's strategy is a control input sequence $U = [u_{t+1}, u_{t+2}, ...,u_{t+N}]$ and $u_{t+1}$ will be applied in each loop. 

The vehicle dynamic is shown in this figure.
<img src=motion_model.png width="200px" >

We define our motion model as follows:
$$
\begin{eqnarray}
\dot{x} &=& v cos(\phi) \\
\dot{y} &=& v sin(\phi)\\
\dot{v} &=& a\\
\dot{\phi} &=& \frac{v}{L} tan(\delta)\\
\dot{\omega} &=&  a sin(\phi-\delta)\\
\dot{l}&=&  a cos(\phi-\delta)\\
\dot{a} &=& a dt\\
\ddot{a} &=& \dot{a} dt\\
\end{eqnarray}
$$
where $L$ is the length between the front and reer wheel.

It stems from executing the control strategies of all the surrouding vehicles on a joint dyanamical systems.
$$ z_{t+1} = A z_{t} + B u_t +C \\
\dot{z} = f(z,u) = A'z + B'u  
$$

ODE can be witten as
$$
\frac{\partial f}{\partial z} =
A'= 
\begin{bmatrix}
0&0&-v sin(\phi)&cos(\phi)&0&0&0&0\\
0&0& v cos(\phi)&sin(\phi)&0&0&0&0\\
0&0& 0&0&0&0&0&0\\
0&0& 0&tan{\delta}/L&0&0&0&0\\
0&0& a cos(\phi-\delta)&0&0&0&0&0\\
0&0& -a sin(\phi-\delta)&0&0&0&0&0\\
0&0& 0&0&0&0&0&0\\
0&0& 0&0&0&0&1&0
\end{bmatrix}
$$

$$
\frac{\partial f}{\partial u} =
B' = 
\begin{bmatrix}
0&0\\
0&0\\
0&1\\
\frac{v}{L cos^2(\delta)}&0\\
-a cos(\phi-\delta)&sin(\phi-\delta)\\
-a sin(\phi-\delta)&cos(\phi-\delta)\\
0&dt\\
0&0\\
\end{bmatrix}
$$

We can get a discrete-time mode with Forward Euler Discretization with sampling time $dt$.

$$z_{t+1} = z_k + f(z_t, u_t)dt$$

Using first degree Tayler expantion around $\bar{z}$ and $\bar{u}$
$$z_{t+1} = z_t + (f({\bar{z}, \bar{u}}) + A'z_t + B'u_t-A'\bar{z}-B'\bar{u})dt$$
$$z_{t+1} = (I+dtA')z_t + (dtB')u_t + (f(\bar{z},\bar{u})-A'\bar{z}-B'\bar{u}))dt$$

Therefore, we get
$$
A= 
\begin{bmatrix}
1&0&-v sin(\phi)&cos(\phi)&0&0&0&0\\
0&1& v cos(\phi)&sin(\phi)&0&0&0&0\\
0&0& 1&0&0&0&0&0\\
0&0& 0&1+tan{\delta}/L&0&0&0&0\\
0&0& a cos(\phi-\delta)&0&1&0&0&0\\
0&0& -a sin(\phi-\delta)&0&0&1&0&0\\
0&0& 0&0&0&&1&0\\
0&0& 0&0&0&0&1&1
\end{bmatrix}
$$

$$
B = 
\begin{bmatrix}
0&0\\
0&0\\
0&dt\\
dt\frac{v}{L*cos^2(\delta)}&0\\
-a*cos(\phi-\delta)dt&sin(\phi-\delta)dt\\
-a*sin(\phi-\delta)dt&cos(\phi-\delta)dt\\
0&dt\\
0&0\\
\end{bmatrix}
$$

$$
C = (f(\bar{z},\bar{u})-A'\bar{z}-B'\bar{u})dt = \\
\begin{bmatrix}
\bar{v} cos(\bar{\phi}) \\
\bar{v} sin(\bar{\phi})\\
\bar{a}\\
\frac{\bar{v}}{L} tan(\bar{\delta})\\
\bar{a}sin(\bar{\phi}-\bar{\delta})\\
\bar{a}cos(\bar{\phi}-\bar{\delta})\\
\bar{a}dt\\
\bar{\dot{a}}dt\\
\end{bmatrix}
-
\begin{bmatrix}
0&0&-v sin(\phi)&cos(\phi)&0&0&0&0\\
0&0& v cos(\phi)&sin(\phi)&0&0&0&0\\
0&0& 0&0&0&0&0&0\\
0&0& 0&tan{\delta}/L&0&0&0&0\\
0&0& a cos(\phi-\delta)&0&0&0&0&0\\
0&0& -a sin(\phi-\delta)&0&0&0&0&0\\
0&0& 0&0&0&0&0&0\\
0&0& 0&0&0&0&1&0
\end{bmatrix}
\begin{bmatrix}\bar{x}\\ \bar{y}\\ \bar{\phi}\\ \bar{v}\\ \bar{\omega}\\  \bar{l} \\ \bar{\dot{a}} \\ \bar{\ddot{a}}\end{bmatrix}
$$

$$-
\begin{bmatrix}
0&0\\
0&0\\
0&1\\
\frac{v}{L*cos^2(\delta)}&0\\
-a*cos(\phi-\delta)&sin(\phi-\delta)\\
-a*sin(\phi-\delta)&cos(\phi-\delta)\\
0&dt\\
0&0\\
\end{bmatrix}
\begin{bmatrix}\bar{\delta}\\\bar{a}\end{bmatrix}$$


$$=
\begin{bmatrix}
\bar{v} cos(\bar{\phi}) \\
\bar{v} sin(\bar{\phi})\\
\bar{a}\\
\frac{\bar{v}}{L} tan(\bar{\delta})\\
\bar{a}sin(\bar{\phi}-\bar{\delta})\\
\bar{a}cos(\bar{\phi}-\bar{\delta})\\
\bar{a}dt\\
\bar{\dot{a}}dt\\
\end{bmatrix} -
\begin{bmatrix}
-\bar{v}*sin(\bar{\phi})\bar{\phi} +\bar{v}cos(\bar{\phi})\\
 \bar{v}*cos(\bar{\phi})\bar{\phi} +\bar{v}sin(\bar{\phi})\\
0\\
tan({\bar\delta})\bar{v}/L\\
\bar{a}cos({\bar{\phi}-\bar{\delta}})\bar{\phi}\\
-\bar{a}sin({\bar{\phi}-\bar{\delta}})\bar{\phi} \\
0\\
0
\end{bmatrix}
$$

$$
-
\begin{bmatrix}
0\\
0\\
\bar{a}\\
\frac{\bar{v}}{L*cos^2(\bar{\delta})}\bar{\delta}\\
\bar{v}\bar{\delta}cos(\bar{\phi}-\bar{\delta})\bar{\delta} + sin(\bar{\phi}-\bar{\delta})\bar{a} \\
-\bar{v}\bar{\delta}sin(\bar{\phi}-\bar{\delta})\bar{\delta}+ cos(\bar{\phi}-\bar{\delta})\bar{a}\\
\bar{a}dt\\
0
\end{bmatrix}
$$


$$
=
\begin{bmatrix}
\bar{v}*sin(\bar{\phi})\bar{\phi} \\
-\bar{v}*cos(\bar{\phi})\bar{\phi}\\
0\\
-\frac{\bar{v}}{L*cos^2(\bar{\delta})}\bar{\delta}\\
\bar{\delta}\bar{a}cos({\bar{\phi}-\bar{\delta}})\bar{\phi}-\bar{v}\bar{\delta}cos(\bar{\phi}-\bar{\delta})\bar{\delta}\\
-\bar{\delta}\bar{a}sin({\bar{\phi}-\bar{\delta}}\bar{\phi}+\bar{v}\bar{\delta}sin(\bar{\phi}+ \bar{\delta})\bar{\delta} \\
0\\
\bar{\dot{a}}dt\\
\end{bmatrix}
$$

$$
C = \begin{bmatrix}
\bar{v}*sin(\bar{\phi})\bar{\phi} \\
-\bar{v}*cos(\bar{\phi})\bar{\phi}\\
0\\
-\frac{\bar{v}}{L*cos^2(\bar{\delta})}\bar{\delta}\\
-\bar{a}cos({\bar{\phi}-\bar{\delta}})\bar{\phi}-\bar{v}\bar{\delta}cos(\bar{\phi}-\bar{\delta})\bar{\delta}\\
\bar{a}sin({\bar{\phi}-\bar{\delta}})\bar{\phi}+\bar{v}\bar{\delta}sin(\bar{\phi}+ \bar{\delta})\bar{\delta} \\
0\\
\bar{\dot{a}}dt\\
\end{bmatrix}dt
$$


### Cost function Parameterization

In order to lean from the feature importance

- $\rho_1,...,\rho_6 $ for state coefficients, with $\sum(\rho_1+...,+\rho_6) = 1$, are obtained from RF model
- $\rho_T $ for terminal state coefficients
- $\theta_1, \theta_2$ for the control coefficients
- $\kappa_1, ..., \kappa_6$ for coefficients of the surrounding vehicle direction.


Therefore, the cost function of Risk Sensative Control will be as following

- Cost function of ego vehicle state
$$
J = \rho_T((x_{t,T} - X_T)^2 + (y_{t,T} - y_T)^2) + \rho_1 \sum_{t=1}^N ((x_t- x_{t,r})^2 + (y_t- y_{t,r})^2)  \\
+ \rho_2  \sum_{t=1}^N (v_t - v_r)^2 \\
+ \rho_2  \sum_{t=1}^N (\phi_t - \phi_{t,r})\\
+ \rho_3  \sum_{t=1}^N (\omega_t - \phi_{\omega,r}) \\
+ \rho_4  \sum_{t=1}^N (l_t - \phi_{l,r}) \\
+ \rho_5  \sum_{t=1}^N (\dot{a}_t - \dot{a}_{l,r}) \\
+ \rho_6  \sum_{t=1}^N (\ddot{a}_t - \ddot{a}_{l,r})
$$

where $(x_{t,T}, y_{t,T})$ is the goal of the target trajectory. $z_{t,r}$ is the referenced state at time step $t$ which obained from real driving data.


- Cost function of ego vehicle control
$$
J += \theta_1\sum_{t=1}^N ((a_t-a_{t-1})^2 + a_t) +\theta_2\sum_{t=1}^N ((\delta_t-\delta_{t-1})^2 + \delta_t)
$$



- Cost function of surrounding vehicle direction
$$
J += \sum_{k=1}^K \kappa_{k}(1/ \sum_{t=1}^N (x_t - x_t^k)^2 + (y_t - y_t^k)^2)
$$

We donote $z_t^k, k \in {1,...,6}$ to represent the state of surrounding vehicle in $k$ direction relative to ego vehicle in $t$ time step.




### Constraints 

- Distance to surrounding vehicles are set to be > 20m.
- $v_{min} < v_t < v_{max}$
- $a_{min} < a_t < a_{max}$
- $\delta_{min} < \delta_t < \delta_{max}$


### Optimization using Sequential Least Squares Programming (SLSQP)
- Optimization method of nonlinear constraints
- Recursive Least Square Method
- ref: [https://docs.scipy.org/doc/scipy/reference/optimize.minimize-slsqp.html#optimize-minimize-slsqp]
